In [6]:
import os.path
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
import seaborn as sns

from pathlib import Path
from tqdm import tqdm
from time import perf_counter

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report,accuracy_score
from IPython.display import Markdown, display

import efficientnet.keras as efn
from keras import backend as K

In [2]:
dir_ = Path('./final_test')
filepaths = list(dir_.glob(r'**/*'))
def proc_img(filepath):
    """
   		이미지데이터의 경로와 label데이터로 데이터프레임 만들기 
    """

    labels = [str(filepath[i]).split("/")[-2] \
              for i in range(len(filepath))]

    filepath = pd.Series(filepath, name='Filepath').astype(str)
    labels = pd.Series(labels, name='Label')

    # 경로와 라벨 concatenate
    test_df3 = pd.concat([filepath, labels], axis=1)

    # index 재설정
    test_df3 = test_df3.sample(frac=1,random_state=0).reset_index(drop = True)
    
    return test_df3

test_df3 = proc_img(filepaths)
test_df3.head(5)

,Filepath,Label
0,final_test/good/test12.jpg,good
1,final_test/bad/starbucks_47_bad.JPG,bad
2,final_test/good/test14.jpg,good
3,final_test/good/test10.jpg,good
4,final_test/good/starbucks_08_good.JPG,good


In [3]:
spac = test_df3[test_df3['Label'].str.contains('final_test')].index

test_df3.drop(spac, inplace=True)

print(f'Number of pictures: {test_df3.shape[0]}\n')
print(f'Number of different labels: {len(test_df3.Label.unique())}\n')
print(f'Labels: {test_df3.Label.unique()}')

Number of pictures: 45

Number of different labels: 2

Labels: ['good' 'bad']


In [4]:
def create_gen():
    # 생성기 및 데이터 증강으로 이미지 로드
    train_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input,
        validation_split=0.1
    )

    test_generator = tf.keras.preprocessing.image.ImageDataGenerator(
        preprocessing_function=tf.keras.applications.mobilenet_v2.preprocess_input
    )

    test_images = test_generator.flow_from_dataframe(
        dataframe=test_df3,
        x_col='Filepath',
        y_col='Label',
        target_size=(448, 448),
        color_mode='rgb',
        class_mode='categorical',
        batch_size=32,
        shuffle=False
    )
    
    return test_generator,test_images

In [5]:
from tensorflow.keras.models   import Sequential , clone_model , Model, load_model

In [14]:
print('1번 모델 복원')
loaded_model = load_model('brand_recog_model_0618.h5')
loaded_model.summary()

1번 모델 복원
Model: "model_8"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_9 (InputLayer)           [(None, 448, 448, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 454, 454, 3)  0           ['input_9[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 224, 224, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                   

 conv2_block2_out (Add)         (None, 112, 112, 25  0           ['conv2_block1_out[0][0]',       
                                6)                                'conv2_block2_3_conv[0][0]']    
                                                                                                  
 conv2_block3_preact_bn (BatchN  (None, 112, 112, 25  1024       ['conv2_block2_out[0][0]']       
 ormalization)                  6)                                                                
                                                                                                  
 conv2_block3_preact_relu (Acti  (None, 112, 112, 25  0          ['conv2_block3_preact_bn[0][0]'] 
 vation)                        6)                                                                
                                                                                                  
 conv2_block3_1_conv (Conv2D)   (None, 112, 112, 64  16384       ['conv2_block3_preact_relu[0][0]'
          

 conv3_block2_1_relu (Activatio  (None, 56, 56, 128)  0          ['conv3_block2_1_bn[0][0]']      
 n)                                                                                               
                                                                                                  
 conv3_block2_2_pad (ZeroPaddin  (None, 58, 58, 128)  0          ['conv3_block2_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv3_block2_2_conv (Conv2D)   (None, 56, 56, 128)  147456      ['conv3_block2_2_pad[0][0]']     
                                                                                                  
 conv3_block2_2_bn (BatchNormal  (None, 56, 56, 128)  512        ['conv3_block2_2_conv[0][0]']    
 ization)                                                                                         
          

                                                                                                  
 conv4_block1_preact_bn (BatchN  (None, 28, 28, 512)  2048       ['conv3_block4_out[0][0]']       
 ormalization)                                                                                    
                                                                                                  
 conv4_block1_preact_relu (Acti  (None, 28, 28, 512)  0          ['conv4_block1_preact_bn[0][0]'] 
 vation)                                                                                          
                                                                                                  
 conv4_block1_1_conv (Conv2D)   (None, 28, 28, 256)  131072      ['conv4_block1_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_blo

 conv4_block3_2_pad (ZeroPaddin  (None, 30, 30, 256)  0          ['conv4_block3_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv4_block3_2_conv (Conv2D)   (None, 28, 28, 256)  589824      ['conv4_block3_2_pad[0][0]']     
                                                                                                  
 conv4_block3_2_bn (BatchNormal  (None, 28, 28, 256)  1024       ['conv4_block3_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_block3_2_relu (Activatio  (None, 28, 28, 256)  0          ['conv4_block3_2_bn[0][0]']      
 n)                                                                                               
          

                                                                                                  
 conv4_block6_preact_relu (Acti  (None, 28, 28, 1024  0          ['conv4_block6_preact_bn[0][0]'] 
 vation)                        )                                                                 
                                                                                                  
 conv4_block6_1_conv (Conv2D)   (None, 28, 28, 256)  262144      ['conv4_block6_preact_relu[0][0]'
                                                                 ]                                
                                                                                                  
 conv4_block6_1_bn (BatchNormal  (None, 28, 28, 256)  1024       ['conv4_block6_1_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv4_blo

 conv5_block2_2_pad (ZeroPaddin  (None, 16, 16, 512)  0          ['conv5_block2_1_relu[0][0]']    
 g2D)                                                                                             
                                                                                                  
 conv5_block2_2_conv (Conv2D)   (None, 14, 14, 512)  2359296     ['conv5_block2_2_pad[0][0]']     
                                                                                                  
 conv5_block2_2_bn (BatchNormal  (None, 14, 14, 512)  2048       ['conv5_block2_2_conv[0][0]']    
 ization)                                                                                         
                                                                                                  
 conv5_block2_2_relu (Activatio  (None, 14, 14, 512)  0          ['conv5_block2_2_bn[0][0]']      
 n)                                                                                               
          

In [15]:
dir_ = Path('./final_models/goodbad')
model_filepaths = list(dir_.glob(r'**/*.h5'))
def proc_img2(model_filepath):
    """
    """

#     labels = [str(filepath[i]).split("/")[-2] \
#               for i in range(len(filepath))]

    model_filepath = pd.Series(model_filepath, name='Filepath').astype(str)
#     labels = pd.Series(labels, name='Label')

    # 경로와 라벨 concatenate
    model_df = pd.DataFrame(model_filepath)
#     test_df3 = pd.concat([filepath, labels], axis=1)

    # index 재설정
    model_df = model_df.sample(frac=1,random_state=0).reset_index(drop = True)
    
    return model_df

model_df = proc_img2(model_filepaths)
model_df



,Filepath
0,final_models/goodbad/good_bad_v3_adam1_epoch30.h5
1,final_models/goodbad/AI_good_bad_0627_MNV2_Dro...
2,final_models/goodbad/good_bad_0623_mbnv3.h5
3,final_models/goodbad/good_bad_shufflenet_adam_...
4,final_models/goodbad/good_bad_xception_adam_ep...
5,final_models/goodbad/good_bad_v3_adam1_epoch20.h5
6,final_models/goodbad/good_bad_0622_mbn.h5


In [16]:
model_list = model_df.values.tolist()

print('2번 모델 복원')
# loaded_model = tf.keras.models.load_model('brand_shufflenet_adam_epoch20_batch16.h5')
for i in model_list:
    print(i[0])
    mod_path = './'+i[0]
    loaded_model = tf.keras.models.load_model(mod_path, custom_objects={'tf': tf})
#     loaded_model.summary()

    test_generator,test_images=create_gen()



    test_images.class_indices

    # Predict the label of the test_images
    pred = loaded_model.predict(test_images)
    pred = np.argmax(pred,axis=1)

    # Map the label
    labels = (test_images.class_indices)
    labels = dict((v,k) for k,v in labels.items())
    pred = [labels[k] for k in pred]

    y_test = list(test_df3.Label)
    acc = accuracy_score(y_test,pred)
    print(f'Accuracy on the test set: {acc * 100:.2f}%')

    class_report = classification_report(y_test, pred, zero_division=1)
    print(class_report)

2번 모델 복원
final_models/goodbad/good_bad_v3_adam1_epoch30.h5
Found 45 validated image filenames belonging to 2 classes.


2022-06-27 13:34:56.355418: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Accuracy on the test set: 73.33%
              precision    recall  f1-score   support

         bad       0.65      0.95      0.77        21
        good       0.93      0.54      0.68        24

    accuracy                           0.73        45
   macro avg       0.79      0.75      0.73        45
weighted avg       0.80      0.73      0.72        45

final_models/goodbad/AI_good_bad_0627_MNV2_Dropout2_epoch20.h5
Found 45 validated image filenames belonging to 2 classes.


2022-06-27 13:35:08.595664: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Accuracy on the test set: 88.89%
              precision    recall  f1-score   support

         bad       0.83      0.95      0.89        21
        good       0.95      0.83      0.89        24

    accuracy                           0.89        45
   macro avg       0.89      0.89      0.89        45
weighted avg       0.90      0.89      0.89        45

final_models/goodbad/good_bad_0623_mbnv3.h5
Found 45 validated image filenames belonging to 2 classes.


...

Accuracy on the test set: 86.67%
              precision    recall  f1-score   support

         bad       1.00      0.71      0.83        21
        good       0.80      1.00      0.89        24

    accuracy                           0.87        45
   macro avg       0.90      0.86      0.86        45
weighted avg       0.89      0.87      0.86        45

final_models/goodbad/good_bad_v3_adam1_epoch20.h5
Found 45 validated image filenames belonging to 2 classes.


2022-06-27 13:35:36.186318: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Accuracy on the test set: 73.33%
              precision    recall  f1-score   support

         bad       0.64      1.00      0.78        21
        good       1.00      0.50      0.67        24

    accuracy                           0.73        45
   macro avg       0.82      0.75      0.72        45
weighted avg       0.83      0.73      0.72        45

final_models/goodbad/good_bad_0622_mbn.h5
Found 45 validated image filenames belonging to 2 classes.


2022-06-27 13:35:41.783351: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:113] Plugin optimizer for device_type GPU is enabled.


Accuracy on the test set: 84.44%
              precision    recall  f1-score   support

         bad       0.82      0.86      0.84        21
        good       0.87      0.83      0.85        24

    accuracy                           0.84        45
   macro avg       0.84      0.85      0.84        45
weighted avg       0.85      0.84      0.84        45

